In [2]:
import xarray as xr
import rioxarray
import os
import glob
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterio import features
import rasterio
import os
from shapely.geometry import shape, box, mapping
from shapely.geometry.polygon import orient
import json
import xclim.indicators.atmos as xci
import sys
from collections import defaultdict

stats = ["max", "min", "med"]
variants = ["tas", "tasmas", "tasmin"]
rolling_period = 30
output_folder = "./processed/tas"
input_folder = "./raw/tas"
models = ["01", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "15", "23", "25", "27", "29"]

In [3]:
def fix_metric(metric, merged_ds):
    metric = metric.groupby('time.year').mean(dim='time', keep_attrs=True)
    metric = metric.isel(year=slice(1, 6))
    metric = metric.transpose(*merged_ds['min'].dims)
    metric = metric.assign_coords({
            "projection_y_coordinate": merged_ds.projection_y_coordinate,
            "projection_x_coordinate": merged_ds.projection_x_coordinate,
            "year": merged_ds.year
        })
    metric.attrs.update(merged_ds["min"].attrs)
    return metric

In [4]:
model_number = "01"

tas_files = glob.glob(os.path.join(input_folder,model_number, "tas_*.nc"))
tasmax_files = glob.glob(os.path.join(input_folder,model_number, "tasmax_*.nc"))
tasmin_files = glob.glob(os.path.join(input_folder,model_number, "tasmin_*.nc"))
all_tas = tas_files + tasmax_files + tasmin_files
combined_ds = xr.open_mfdataset(all_tas, combine='by_coords')

yearly_groups = []
for i, (year, yearly_ds) in enumerate(combined_ds.groupby('time.year')):
    if i > 5:
        break
    augmented_yearly_data = yearly_ds.mean(dim='time', keep_attrs=True)        
    augmented_yearly_data["min"] = yearly_ds['tasmin'].min(dim='time', keep_attrs=True)
    augmented_yearly_data["max"] = yearly_ds['tasmax'].max(dim='time', keep_attrs=True)
    yearly_groups.append(augmented_yearly_data)
    
yearly_groups = yearly_groups[1:]
merged_ds = xr.concat(yearly_groups, dim='year')
combined_ds.close()

cdda=xci.cooling_degree_days_approximation(
    tas=combined_ds['tas'], 
    tasmax=combined_ds['tasmax'],
    tasmin=combined_ds['tasmin'],
    freq='YS',
    thresh="20.0 degC"
)
merged_ds['cooling_degree_days'] = fix_metric(cdda, merged_ds)

hdda=xci.heating_degree_days_approximation(
    tas=combined_ds['tas'], 
    tasmax=combined_ds['tasmax'],
    tasmin=combined_ds['tasmin'],
    freq='YS',
    thresh="15.5 degC"
)
merged_ds['heating_degree_days'] = fix_metric(hdda, merged_ds)

hwf = xci.heat_wave_frequency(
    tasmax=combined_ds['tasmax'],
    tasmin=combined_ds['tasmin'],
    freq='YS',
    thresh_tasmax="26.5 degC",
    thresh_tasmin='22.0 degC',
    window=3
)
merged_ds['heatwave_freq'] = fix_metric(hwf, merged_ds)

hwl = xci.heat_wave_max_length(
    tasmax=combined_ds['tasmax'],
    tasmin=combined_ds['tasmin'],
    freq='YS',
    thresh_tasmax="26.5 degC",
    thresh_tasmin='22.0 degC',
)
merged_ds['heatwave_length'] = fix_metric(hwl, merged_ds)

cfd = xci.consecutive_frost_days(
    tasmin=combined_ds['tasmin'],
)

merged_ds['conseq_frost_days'] = fix_metric(cfd, merged_ds)

merged_ds = merged_ds.drop_vars(['tas', 'tasmax', 'tasmin'])
out_rolling_path = os.path.join(output_folder, '01_rolling', f"rolling_average_{model_number}.nc")
merged_ds.to_netcdf(out_rolling_path)

c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:77: UserWarning: Variable has a non-conforming cell_methods: Got `time: mean`, which do not include the expected `time: maximum`.
  _check_cell_methods(getattr(vardata, "cell_methods", None), data["cell_methods"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:77: UserWarning: Variable has a non-conforming cell_methods: Got `time: mean`, which do not include the expected `time: minimum`.
  _check_cell_methods(getattr(vardata, "cell_methods", None), data["cell_methods"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:77: UserWarning: Variable has a non-conforming cell_methods: Got `time: mean`, which do not include the expected `time: maximum`.
  _check_cell_methods(getattr(vardata, "cell_methods", None), data["cell_methods"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim